# TensorFlow Core tutorial

In [2]:
import tensorflow as tf

# Create Constant

In [3]:
node1=tf.constant(3.0,dtype=tf.float32)
node2=tf.constant(4.0)
print(node1,node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
# create a session object and then invoke its run method to evaluate node1 and node2
sess=tf.Session()
print(sess.run([node1,node2]))

[3.0, 4.0]


In [5]:
## cobine tensor nodes with operation
node3=tf.add(node1,node2)
print('node3:',node3)
print('sess.run(node3):',sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [6]:
# placeholder
a=tf.placeholder(tf.float32)
b=tf.placeholder(tf.float32)
adder_node=a+b # + provides a shortcut for tf.add(a,b)

In [7]:
print(sess.run(adder_node,{a:3,b:4.5}))
print(sess.run(adder_node,{a:[1,3],b:[2,4]}))

7.5
[3. 7.]


In [8]:
## add another operation
add_and_triple=adder_node*3
print(sess.run(add_and_triple,{a:3,b:4.5}))

22.5


# Create Varaibles

In [17]:
W=tf.Variable([.3],dtype=tf.float32)
b=tf.Variable([-.3],dtype=tf.float32)
x=tf.placeholder(tf.float32)
linear_model=W*x+b

In [18]:
# constants are initialized when you call tf.const, and their value can never change.
# variables are not initialized when you call tf.Variable. 
# you need to initialize all the variables in a tensorflow program.
init=tf.global_variables_initializer()
sess.run(init)

In [19]:
print(sess.run(linear_model,{x:[1,2,3,4]}))

[0.         0.3        0.6        0.90000004]


# Lost Function

In [20]:
y=tf.placeholder(tf.float32)
squared_deltas=tf.square(linear_model-y)
loss=tf.reduce_sum(squared_deltas)
print(sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

23.66


In [21]:
print('Before sess.run(W):',sess.run(W))
print('Before sess.run(b):',sess.run(b))
print('\n')
fixW=tf.assign(W,[-1.0])
fixb=tf.assign(b,[1.])
sess.run([fixW,fixb])
print('After sess.run(W):',sess.run(W))
print('After sess.run(b):',sess.run(b))

Before sess.run(W): [0.3]
Before sess.run(b): [-0.3]


After sess.run(W): [-1.]
After sess.run(b): [1.]


In [22]:
print(sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

0.0


# tf.train API

In [27]:
# this is a test cell 
testA=tf.Variable([3],dtype=tf.int32)
testB=tf.Variable([5],dtype=tf.float32)
init=tf.global_variables_initializer()
sess.run(init)
print(sess.run(testA))
print(sess.run(testB))

print(sess.run(tf.assign(testA,[33])))
print(sess.run(tf.assign(testB,[55])))

[3]
[5.]
[33]
[55.]


In [28]:
# this is also a test cell
print(sess.run(testA))
print(sess.run(testB))

[33]
[55.]


In [29]:
optimizer=tf.train.GradientDescentOptimizer(0.01)
train=optimizer.minimize(loss)

In [30]:
sess.run(init)  # reset values to incorrect defaults
for i in range(1000):
    sess.run(train,{x:[1,2,3,4],y:[0,-1,-2,-3]})  # x and y are the placeholder of variable train
print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([0.9999908], dtype=float32)]


# Complete Program

In [31]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [0.9999908] loss: 5.6999738e-11


# tf.estimator

In [36]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

AttributeError: module 'tensorflow' has no attribute 'numeric_column'

In [37]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_save_summary_steps': 100, '_tf_random_seed': 1, '_model_dir': 'C:\\Users\\computer\\AppData\\Local\\Temp\\tmp3mnd9rud'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\computer\AppData\Local\Temp\tmp3mnd9rud\model.ckpt.
INFO:tensorflow:loss = 5.312389240806137, step = 1
INFO:tensorflow:global_step/sec: 1063.78
INFO:tensorflow:loss = 0.023279510970789126, step = 101 (0.095 sec)
INFO:tensorflow:global_step/sec: 1176.4
INFO:tensorflow:loss = 0.001790437566807422, step = 201 (0.086 sec)
INFO:tensorflow:global_step/sec: 1176.4
INFO:tensorflow:loss = 0.0003680308147527537, step = 301 (0.084 sec)
INFO:tensorflow:global_step/sec: 1176.4
INFO:tensorflow:loss = 3.484089232859266e-05, step = 401 (0.085 sec)
INFO:tensorflow